# Common Core

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/gps-wizard/

In [1]:
import os
import sys

from datetime import date, datetime, timedelta

import unittest

import csv
import numpy as np

projdir = os.path.realpath(os.path.join(sys.path[0], ".."))

## Common Definitions

e.g. Nation and region names

In [2]:
verbose = False

## Printable Class

Simple class that allows other classes to be printed.

In [3]:
class Printable:
    def __repr__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

## Load CSV into NumPy Array

SImple function to load data from CSV into an ndarray

In [4]:
def loadCsvIntoArray(fileName, verbose=verbose):
    '''Load CSV file into numpy array'''

    if verbose:
        partName = os.path.basename(fileName)

        print(f"Loading {partName}...")

    try:
        with open(fileName, 'r') as f:
            reader = csv.reader(f, delimiter = ',')

            dtype = []
            converters = {}
            colNames = next(reader)

            for i in range(len(colNames)):
                colName = colNames[i]
                if colName in ["distance", "speed", "hdop", "cog"]:
                    dtype.append((colName, "f4"))
                elif colName in ["position_lat", "position_long"]:
                    dtype.append((colName, "i4"))
                else:
                    dtype.append((colName, "u4"))

            data = np.genfromtxt(f, dtype=dtype, delimiter=",")

    except:
        print(f"Failed to load {fileName}")
        raise
        
    return data

## NumPy Helper Functions

Useful functionality such as moving average or rolling sum

In [5]:
def movingAverage(data, window=7):
    """Calculate moving average using linear convolution"""

    # Only use convolution if the input is at least as long as the window size
    if len(data) >= window:
        # The mode "valid" results in the less values than required, hence the np.zeros()
        result = np.concatenate((np.zeros(window // 2),
                                 np.convolve(data, np.ones(window) / window, mode="valid"),
                                 np.zeros(window // 2)))

    else:
        # Result is a simple ndarray of zeros
        result = np.zeros(len(data))

    return result

In [6]:
class TestMovingAverage(unittest.TestCase):
    '''Class to test rollingSum function'''   

    def testShortList(self):
        '''Test processing of a list shorter than the window size'''

        actual = movingAverage(np.arange(6), 7)
        expected = np.array(np.zeros(6))

        self.assertEqual((actual == expected).all(), True)


    def testLongerList(self):
        '''Test processing of a list longer than the window size'''

        actual = movingAverage(np.arange(14), 7)
        expected = np.array([0, 0, 0, 3, 4, 5, 6, 7, 8, 9, 10, 0, 0, 0])

        # Comparison of floating point values must be approximate
        self.assertEqual((abs(actual - expected) < 1e-10).all(), True)

## Run Unit Tests

In [7]:
if __name__ == '__main__':
    import __main__ as main
    exit = hasattr(main, '__file__')
    unittest.main(argv=['first-arg-is-ignored'], exit=exit)

..
----------------------------------------------------------------------
Ran 2 tests in 0.001s

OK
